In [1]:
import pandas as pd
import numpy as np

from numpy import meshgrid

## For plotting
import matplotlib.pyplot as plt
from matplotlib import style
import datetime as dt
import seaborn as sns
sns.set_style("whitegrid")

In [2]:
df= pd.read_csv('df7.csv', parse_dates=['Date'])
df = df.rename(columns = {"Date":"ds","Close":"y"}) 
df 

,ds,y,tby,ffr,fta,eps,div,une,vix
0,2005-06-20,1216.10,4.11,3.04,812235.0,84.81,1.76,5.0,148.30
1,2005-06-21,1213.61,4.06,3.04,812235.0,84.81,1.76,5.0,149.00
2,2005-06-22,1213.88,3.95,3.04,811179.0,84.81,1.76,5.0,148.40
3,2005-06-23,1200.73,3.96,3.04,811179.0,84.81,1.76,5.0,150.50
4,2005-06-24,1191.57,3.92,3.04,811179.0,84.81,1.76,5.0,152.00
...,...,...,...,...,...,...,...,...,...
3893,2021-01-25,3855.36,1.05,0.09,7414942.0,95.72,1.58,6.3,25.85
3894,2021-01-26,3849.62,1.05,0.09,7414942.0,95.72,1.58,6.3,25.30
3895,2021-01-27,3750.77,1.04,0.09,7404926.0,95.72,1.58,6.3,33.05
3896,2021-01-28,3787.38,1.07,0.09,7404926.0,95.72,1.58,6.3,30.49


In [3]:
from datetime import datetime
p = 0.9
# Train around 90% of dataset 
# cutoff = int((p*len(df)//100)*100)
cutoff = int(p*len(df))
df_train = df[:cutoff].copy()
df_test = df.drop(df_train.index).copy()

print(df_train, df_test)

             ds        y   tby   ffr        fta     eps   div  une     vix
0    2005-06-20  1216.10  4.11  3.04   812235.0   84.81  1.76  5.0  148.30
1    2005-06-21  1213.61  4.06  3.04   812235.0   84.81  1.76  5.0  149.00
2    2005-06-22  1213.88  3.95  3.04   811179.0   84.81  1.76  5.0  148.40
3    2005-06-23  1200.73  3.96  3.04   811179.0   84.81  1.76  5.0  150.50
4    2005-06-24  1191.57  3.92  3.04   811179.0   84.81  1.76  5.0  152.00
...         ...      ...   ...   ...        ...     ...   ...  ...     ...
3503 2019-07-02  2973.01  1.98  2.40  3826817.0  139.88  1.94  3.6   14.25
3504 2019-07-03  2995.82  1.96  2.40  3813198.0  139.88  1.94  3.6   14.00
3505 2019-07-05  2990.41  2.04  2.40  3813198.0  139.88  1.94  3.6   14.35
3506 2019-07-08  2975.95  2.05  2.40  3813198.0  139.88  1.94  3.6   14.85
3507 2019-07-09  2979.63  2.07  2.40  3813198.0  139.88  1.94  3.6   14.90

[3508 rows x 9 columns]              ds        y   tby   ffr        fta     eps   div  une    vix
3

In [ ]:
from sklearn.metrics import mean_absolute_error as MAE

In [ ]:
# pred_size =100 # predicted periods 
# q = 0.4
# mod = len(df_train) % pred_size
# start = int(q * (len(df_train) - mod)) # 1000 # the number of initial data for training 
# pred_size =100 # predicted periods 
# num_winds = int((df_train.shape[0]-start)/pred_size)
# num_winds

In [62]:
int(0.4 * (len(df_train) - len(df_train) % 100)) + len(df_train) % 100

1668

In [65]:
df

,ds,y,tby,ffr,fta,eps,div,une
0,2003-01-02,909.03,4.07,1.24,732202.0,40.40,1.79,5.8
1,2003-01-03,908.59,4.05,1.24,732202.0,40.40,1.79,5.8
2,2003-01-06,929.01,4.09,1.24,732202.0,40.40,1.79,5.8
3,2003-01-07,922.93,4.04,1.24,732202.0,40.40,1.79,5.8
4,2003-01-08,909.93,4.00,1.24,724902.0,40.40,1.79,5.8
...,...,...,...,...,...,...,...,...
4515,2021-01-25,3855.36,1.05,0.09,7414942.0,95.72,1.58,6.3
4516,2021-01-26,3849.62,1.05,0.09,7414942.0,95.72,1.58,6.3
4517,2021-01-27,3750.77,1.04,0.09,7404926.0,95.72,1.58,6.3
4518,2021-01-28,3787.38,1.07,0.09,7404926.0,95.72,1.58,6.3


In [72]:
len(df_train)

4068

In [64]:
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
pred_size = int((1-p)* len(df))
# fbp = Prophet(daily_seasonality=True)
fbp = Prophet(growth='linear',daily_seasonality=False )
fbp.fit(df_train[['ds','y']])
    
 # predict pred_size futures and get the forecast price 
fut = fbp.make_future_dataframe(periods = pred_size,)
forecast = fbp.predict(fut) 
cv_results = cross_validation(fbp, initial='1668 days', period= '100 days', horizon= '1 day')

forecast

INFO:fbprophet:Making 43 forecasts with cutoffs between 2007-09-20 00:00:00 and 2019-04-08 00:00:00


  0%|          | 0/43 [00:00<?, ?it/s]

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2003-01-02,852.245908,761.827073,912.792713,852.245908,852.245908,-15.020505,-15.020505,-15.020505,-10.194672,-10.194672,-10.194672,-4.825833,-4.825833,-4.825833,0.0,0.0,0.0,837.225403
1,2003-01-03,852.922156,762.035332,915.852791,852.922156,852.922156,-14.788671,-14.788671,-14.788671,-10.797616,-10.797616,-10.797616,-3.991054,-3.991054,-3.991054,0.0,0.0,0.0,838.133486
2,2003-01-06,854.950902,760.253880,915.235979,854.950902,854.950902,-13.305864,-13.305864,-13.305864,-11.740234,-11.740234,-11.740234,-1.565630,-1.565630,-1.565630,0.0,0.0,0.0,841.645039
3,2003-01-07,855.627151,765.273895,922.874771,855.627151,855.627151,-10.505134,-10.505134,-10.505134,-9.648960,-9.648960,-9.648960,-0.856173,-0.856173,-0.856173,0.0,0.0,0.0,845.122017
4,2003-01-08,856.303400,765.181304,926.324628,856.303400,856.303400,-10.707236,-10.707236,-10.707236,-10.476549,-10.476549,-10.476549,-0.230687,-0.230687,-0.230687,0.0,0.0,0.0,845.596164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4514,2020-06-29,3275.108287,3035.441343,3519.083744,3067.912119,3523.272227,-15.342281,-15.342281,-15.342281,-11.740234,-11.740234,-11.740234,-3.602047,-3.602047,-3.602047,0.0,0.0,0.0,3259.766006
4515,2020-06-30,3275.880644,3037.486652,3525.788416,3068.006416,3524.974840,-13.279416,-13.279416,-13.279416,-9.648960,-9.648960,-9.648960,-3.630456,-3.630456,-3.630456,0.0,0.0,0.0,3262.601228
4516,2020-07-01,3276.653001,3043.363430,3526.532023,3068.100713,3526.677452,-13.927682,-13.927682,-13.927682,-10.476549,-10.476549,-10.476549,-3.451133,-3.451133,-3.451133,0.0,0.0,0.0,3262.725319
4517,2020-07-02,3277.425358,3055.896934,3512.674320,3067.970002,3528.380065,-13.264224,-13.264224,-13.264224,-10.194672,-10.194672,-10.194672,-3.069551,-3.069551,-3.069551,0.0,0.0,0.0,3264.161134


In [69]:
from sklearn.metrics import mean_absolute_error as MAE
mae = MAE(cv_results.y, cv_results.yhat)

from sklearn.metrics import mean_squared_error as MSE
mse = MSE(cv_results.y, cv_results.yhat)
print(mae,mse)

88.08170415117735 15752.269230497152


In [78]:
cv_results = cross_validation(fbp, initial='1668 days', period= '100 days', horizon='5 days')
mse = MSE(cv_results.y, cv_results.yhat)
print(mae,mse)

INFO:fbprophet:Making 43 forecasts with cutoffs between 2007-10-04 00:00:00 and 2019-04-04 00:00:00


  0%|          | 0/43 [00:00<?, ?it/s]

56.60968363531394 14920.91002486052


In [79]:
cv_results

,ds,yhat,yhat_lower,yhat_upper,y,cutoff
0,2007-10-05,1502.642348,1481.124787,1526.348655,1557.59,2007-10-04
1,2007-10-09,1498.899872,1475.549243,1522.290801,1565.15,2007-10-04
2,2008-01-14,1489.225129,1461.647280,1516.276365,1416.25,2008-01-12
3,2008-01-15,1489.317302,1462.843520,1516.189948,1380.95,2008-01-12
4,2008-01-16,1489.865451,1462.875231,1517.561907,1373.20,2008-01-12
...,...,...,...,...,...,...
137,2018-12-27,2888.025583,2814.210421,2953.628318,2488.83,2018-12-25
138,2018-12-28,2889.706969,2824.853414,2954.337732,2485.74,2018-12-25
139,2019-04-05,2925.027505,2843.563081,3002.919902,2892.74,2019-04-04
140,2019-04-08,2926.943624,2856.057261,3005.376053,2895.77,2019-04-04


In [85]:
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
pred_size = int((1-p)* len(df))
# fbp = Prophet(daily_seasonality=True)
fbp = Prophet(growth='linear',daily_seasonality=False )
fbp.fit(df_train[['ds','y']])   
 # predict pred_size futures and get the forecast price 
fut = fbp.make_future_dataframe(periods= pred_size)
cv_results = cross_validation(fbp, horizon='100 days')
mse = MSE(cv_results.y, cv_results.yhat)
print(mae,mse)

INFO:fbprophet:Making 110 forecasts with cutoffs between 2004-01-28 00:00:00 and 2018-12-30 00:00:00


  0%|          | 0/110 [00:00<?, ?it/s]

56.60968363531394 20366.59655951445


In [97]:
cv_results

,ds,yhat,yhat_lower,yhat_upper,y,cutoff
0,2004-01-29,1127.790479,1115.502333,1140.935575,1134.11,2004-01-28
1,2004-01-30,1126.773574,1113.534729,1138.627374,1131.13,2004-01-28
2,2004-02-02,1126.384981,1113.264936,1139.918873,1135.26,2004-01-28
3,2004-02-03,1127.713612,1115.602305,1140.296338,1136.03,2004-01-28
4,2004-02-04,1127.026499,1113.871487,1140.342988,1126.52,2004-01-28
...,...,...,...,...,...,...
7524,2019-04-03,2960.321514,2886.788454,3036.779686,2873.40,2018-12-30
7525,2019-04-04,2961.099222,2888.211017,3038.395112,2879.39,2018-12-30
7526,2019-04-05,2960.954964,2885.266315,3033.841325,2892.74,2018-12-30
7527,2019-04-08,2962.391051,2886.069937,3039.893440,2895.77,2018-12-30


In [98]:
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
pred_size = int((1-p)* len(df))
# fbp = Prophet(daily_seasonality=True)
fbp = Prophet(growth='linear',daily_seasonality=False )
fbp.fit(df_train[['ds','y']])
    
 # predict pred_size futures and get the forecast price 
fut = fbp.make_future_dataframe(periods = pred_size)
forecast = fbp.predict(fut) 
cv_results = cross_validation(fbp, initial='1000 days', period= '100 days', horizon= '1100 day')
print(MSE(cv_results.y, cv_results.yhat))

INFO:fbprophet:Making 39 forecasts with cutoffs between 2005-11-08 00:00:00 and 2016-04-04 00:00:00


  0%|          | 0/39 [00:00<?, ?it/s]

251032.08916410134


In [ ]:
from fbprophet.diagnostics import generate_cutoffs()


In [70]:
from fbprophet import Prophet
pred_size = int((1-p)* len(df))
# fbp = Prophet(daily_seasonality=True)
fbp = Prophet(growth='linear',daily_seasonality=True )
fbp.fit(df_train[['ds','y']])
    
 # predict pred_size futures and get the forecast price 
fut = fbp.make_future_dataframe(periods = pred_size,)
forecast = fbp.predict(fut) 
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2003-01-02,892.767611,763.256277,907.518154,892.767611,892.767611,-56.473027,-56.473027,-56.473027,-49.512047,...,-2.101035,-2.101035,-2.101035,-4.859945,-4.859945,-4.859945,0.0,0.0,0.0,836.294583
1,2003-01-03,893.449309,758.205746,910.977850,893.449309,893.449309,-56.234827,-56.234827,-56.234827,-49.512047,...,-2.701664,-2.701664,-2.701664,-4.021116,-4.021116,-4.021116,0.0,0.0,0.0,837.214482
2,2003-01-06,895.494404,763.732130,916.843952,895.494404,895.494404,-54.740194,-54.740194,-54.740194,-49.512047,...,-3.643538,-3.643538,-3.643538,-1.584609,-1.584609,-1.584609,0.0,0.0,0.0,840.754210
3,2003-01-07,896.176102,768.768172,921.803054,896.176102,896.176102,-51.936586,-51.936586,-51.936586,-49.512047,...,-1.552676,-1.552676,-1.552676,-0.871863,-0.871863,-0.871863,0.0,0.0,0.0,844.239516
4,2003-01-08,896.857800,765.416000,921.488609,896.857800,896.857800,-52.134450,-52.134450,-52.134450,-49.512047,...,-2.379098,-2.379098,-2.379098,-0.243304,-0.243304,-0.243304,0.0,0.0,0.0,844.723350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4514,2020-06-29,3316.821668,3050.972615,3527.369526,3124.087507,3559.194492,-56.693726,-56.693726,-56.693726,-49.512047,...,-3.643538,-3.643538,-3.643538,-3.538141,-3.538141,-3.538141,0.0,0.0,0.0,3260.127942
4515,2020-06-30,3317.594247,3053.662212,3517.998659,3124.791105,3561.445997,-54.633144,-54.633144,-54.633144,-49.512047,...,-1.552676,-1.552676,-1.552676,-3.568421,-3.568421,-3.568421,0.0,0.0,0.0,3262.961103
4516,2020-07-01,3318.366825,3070.989801,3532.003524,3125.494702,3563.697502,-55.282119,-55.282119,-55.282119,-49.512047,...,-2.379098,-2.379098,-2.379098,-3.390974,-3.390974,-3.390974,0.0,0.0,0.0,3263.084706
4517,2020-07-02,3319.139403,3064.935266,3513.671498,3126.230007,3565.949007,-54.624366,-54.624366,-54.624366,-49.512047,...,-2.101035,-2.101035,-2.101035,-3.011284,-3.011284,-3.011284,0.0,0.0,0.0,3264.515037


In [74]:
len(df)

4520

In [75]:
len(forecast)

4519

In [76]:
from sklearn.metrics import mean_absolute_error as MAE
mae = MAE(df[:len(forecast)].y, forecast.yhat)

from sklearn.metrics import mean_squared_error as MSE
mse = MSE(df[:len(forecast)].y, forecast.yhat)
print(mae,mse)

56.60968363531394 9262.58155279383


In [47]:
df['fbsp'] = np.nan 
df['fbsp'] = forecast.yhat

In [48]:
df=df.dropna()
df['diff'] = df['y'].copy() - df['fbsp'].copy()

<ipython-input-48-c1ef86ecda73>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['diff'] = df['y'].copy() - df['fbsp'].copy()


In [49]:
df['ds'] = pd.to_datetime(df['ds'])
df = df.set_index(df['ds'])

<ipython-input-49-9236c9dcf51b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ds'] = pd.to_datetime(df['ds'])


In [52]:
df = df.drop(columns = 'ds')

In [53]:
df

,y,tby,ffr,fta,eps,div,une,fbsp,diff
ds,,,,,,,,,
2003-01-02,909.03,4.07,1.24,732202.0,40.40,1.79,5.8,836.294583,72.735417
2003-01-03,908.59,4.05,1.24,732202.0,40.40,1.79,5.8,837.214482,71.375518
2003-01-06,929.01,4.09,1.24,732202.0,40.40,1.79,5.8,840.754210,88.255790
2003-01-07,922.93,4.04,1.24,732202.0,40.40,1.79,5.8,844.239516,78.690484
2003-01-08,909.93,4.00,1.24,724902.0,40.40,1.79,5.8,844.723350,65.206650
...,...,...,...,...,...,...,...,...,...
2021-01-22,3841.47,1.10,0.09,7414942.0,95.72,1.58,6.3,3260.127942,581.342058
2021-01-25,3855.36,1.05,0.09,7414942.0,95.72,1.58,6.3,3262.961103,592.398897
2021-01-26,3849.62,1.05,0.09,7414942.0,95.72,1.58,6.3,3263.084706,586.535294


In [54]:
df.to_csv(r'/Users/yuqingdai/Documents/GitHub/Stock-Erdos/scratch work/Yuqing-Data-Merge/dff2.csv')

In [27]:
import pandas as pd
import numpy as np

from numpy import meshgrid

## For plotting
import matplotlib.pyplot as plt
from matplotlib import style
import datetime as dt
import seaborn as sns
sns.set_style("whitegrid")

In [28]:
df2= pd.read_csv('df7.csv', parse_dates=['Date'])
df2 = df2.rename(columns = {"Date":"ds","Close":"y"}) 
df2 

,ds,y,tby,ffr,fta,eps,div,une,vix
0,2005-06-20,1216.10,4.11,3.04,812235.0,84.81,1.76,5.0,148.30
1,2005-06-21,1213.61,4.06,3.04,812235.0,84.81,1.76,5.0,149.00
2,2005-06-22,1213.88,3.95,3.04,811179.0,84.81,1.76,5.0,148.40
3,2005-06-23,1200.73,3.96,3.04,811179.0,84.81,1.76,5.0,150.50
4,2005-06-24,1191.57,3.92,3.04,811179.0,84.81,1.76,5.0,152.00
...,...,...,...,...,...,...,...,...,...
3893,2021-01-25,3855.36,1.05,0.09,7414942.0,95.72,1.58,6.3,25.85
3894,2021-01-26,3849.62,1.05,0.09,7414942.0,95.72,1.58,6.3,25.30
3895,2021-01-27,3750.77,1.04,0.09,7404926.0,95.72,1.58,6.3,33.05
3896,2021-01-28,3787.38,1.07,0.09,7404926.0,95.72,1.58,6.3,30.49


In [29]:
from datetime import datetime
p = 0.9
# Train around 90% of dataset 
# cutoff = int((p*len(df)//100)*100)
cutoff = int(p*len(df2))
df2_train = df2[:cutoff].copy()
df2_test = df2.drop(df2_train.index).copy()

from fbprophet import Prophet
pred_size = int((1-p)* len(df2))
fbp = Prophet(daily_seasonality=True)
# fit close price using fbprophet model
fbp.fit(df2_train[['ds','y']])
    
 # predict pred_size futures and get the forecast price 
fut = fbp.make_future_dataframe(periods = pred_size,)
forecast = fbp.predict(fut) 
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2005-06-20,1206.220427,1128.065721,1290.030791,1206.220427,1206.220427,2.909537,2.909537,2.909537,4.501782,...,-1.358736,-1.358736,-1.358736,-0.233508,-0.233508,-0.233508,0.0,0.0,0.0,1209.129964
1,2005-06-21,1206.431340,1133.815105,1287.820960,1206.431340,1206.431340,4.233639,4.233639,4.233639,4.501782,...,1.180820,1.180820,1.180820,-1.448963,-1.448963,-1.448963,0.0,0.0,0.0,1210.664979
2,2005-06-22,1206.642254,1135.275728,1284.327226,1206.642254,1206.642254,2.228165,2.228165,2.228165,4.501782,...,0.325807,0.325807,0.325807,-2.599423,-2.599423,-2.599423,0.0,0.0,0.0,1208.870419
3,2005-06-23,1206.853167,1135.071909,1283.874521,1206.853167,1206.853167,1.220525,1.220525,1.220525,4.501782,...,0.377832,0.377832,0.377832,-3.659089,-3.659089,-3.659089,0.0,0.0,0.0,1208.073692
4,2005-06-24,1207.064081,1128.490070,1281.649970,1207.064081,1207.064081,0.497711,0.497711,0.497711,4.501782,...,0.599724,0.599724,0.599724,-4.603794,-4.603794,-4.603794,0.0,0.0,0.0,1207.561792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3892,2020-07-28,3221.494132,2973.783237,3479.839456,2960.649779,3471.699809,19.743563,19.743563,19.743563,4.501782,...,1.180820,1.180820,1.180820,14.060961,14.060961,14.060961,0.0,0.0,0.0,3241.237695
3893,2020-07-29,3222.203482,2981.139670,3495.498497,2960.635047,3474.518040,18.127741,18.127741,18.127741,4.501782,...,0.325807,0.325807,0.325807,13.300152,13.300152,13.300152,0.0,0.0,0.0,3240.331223
3894,2020-07-30,3222.912832,3000.375120,3489.115978,2960.153770,3477.336271,17.312609,17.312609,17.312609,4.501782,...,0.377832,0.377832,0.377832,12.432994,12.432994,12.432994,0.0,0.0,0.0,3240.225441
3895,2020-07-31,3223.622183,2977.510731,3506.865571,2959.492773,3480.154502,16.574432,16.574432,16.574432,4.501782,...,0.599724,0.599724,0.599724,11.472927,11.472927,11.472927,0.0,0.0,0.0,3240.196615


In [33]:
del(df2['fbsp'])

In [34]:
df2['fbsp'] = np.nan 
df2['fbsp'] = forecast.trend

<ipython-input-34-3bb76cdcdb1d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['fbsp'] = np.nan
<ipython-input-34-3bb76cdcdb1d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['fbsp'] = forecast.trend


In [35]:
df2=df2.dropna()
df2['diff'] = df2['y'].copy() - df2['fbsp'].copy()

In [37]:
df2

,ds,y,tby,ffr,fta,eps,div,une,vix,diff,fbsp
0,2005-06-20,1216.10,4.11,3.04,812235.0,84.81,1.76,5.0,148.30,9.879573,1206.220427
1,2005-06-21,1213.61,4.06,3.04,812235.0,84.81,1.76,5.0,149.00,7.178660,1206.431340
2,2005-06-22,1213.88,3.95,3.04,811179.0,84.81,1.76,5.0,148.40,7.237746,1206.642254
3,2005-06-23,1200.73,3.96,3.04,811179.0,84.81,1.76,5.0,150.50,-6.123167,1206.853167
4,2005-06-24,1191.57,3.92,3.04,811179.0,84.81,1.76,5.0,152.00,-15.494081,1207.064081
...,...,...,...,...,...,...,...,...,...,...,...
3892,2021-01-22,3841.47,1.10,0.09,7414942.0,95.72,1.58,6.3,24.85,619.975868,3221.494132
3893,2021-01-25,3855.36,1.05,0.09,7414942.0,95.72,1.58,6.3,25.85,633.156518,3222.203482
3894,2021-01-26,3849.62,1.05,0.09,7414942.0,95.72,1.58,6.3,25.30,626.707168,3222.912832
3895,2021-01-27,3750.77,1.04,0.09,7404926.0,95.72,1.58,6.3,33.05,527.147817,3223.622183


In [38]:
df2.to_csv(r'/Users/yuqingdai/Documents/GitHub/Stock-Erdos/scratch work/Yuqing-Data-Merge/dff4.csv')